In [84]:
import pandas as pd
import sys
data = pd.read_csv("titanic/train.csv")
test = pd.read_csv("titanic/test.csv")
test_ids = test["PassengerId"]
data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [85]:
def clean(data):
    data = data.drop(["PassengerId", "Name", "Ticket"], axis=1)

    cols = ["SibSp", "Parch", "Fare", "Age"]
    for col in cols:
        data[col].fillna(data[col].median(), inplace=True)
    data.Embarked.fillna("U", inplace=True)
    #print(data.Cabin.value_counts().idxmax())
    return data
data = clean(data)
test = clean(test)
data.Cabin.fillna("B96", inplace=True)
test.Cabin.fillna("B57", inplace=True)
data.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,B96,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,B96,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,B96,S


In [81]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
le = preprocessing.LabelEncoder()

cols = ["Sex", "Embarked", "Cabin"]
for col in cols:
    data[col] = le.fit_transform(data[col])
    test[col] = le.transform(test[col])
    print(le.classes_)

['female' 'male']
['C' 'Q' 'S' 'U']


ValueError: y contains previously unseen labels: 'B57'

In [86]:
y = data["Survived"]
X = data.drop(["Survived"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [87]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)

In [88]:
model.fit(X_train, y_train)

ValueError: could not convert string to float: 'male'

In [60]:
predictions = model.predict(X_test)

In [61]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
accuracy_score(y_test, predictions)

0.8212290502793296

In [62]:
submission_preds = model.predict(test)

In [63]:
df = pd.DataFrame({"PassengerId": test_ids.values, "Survived": submission_preds})
df.to_csv("submission.csv", index=False)